In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Hapus data lama jika ada (untuk praktikum)
courses_collection.delete_many({})

# Bulk insert of courses with student enrollments
operations = [
    InsertOne({
        'course': 'Math 101', 
        'enrollments': 30, 
        'department': 'Mathematics'
    }),
    InsertOne({
        'course': 'CS 102', 
        'enrollments': 25, 
        'department': 'Computer Science'
    }),
    InsertOne({
        'course': 'History 201', 
        'enrollments': 20, 
        'department': 'History'
    }),
    InsertOne({
        'course': 'Physics 202', 
        'enrollments': 15, 
        'department': 'Physics'
    })
]

result = courses_collection.bulk_write(operations)
print(f'Courses inserted successfully. Total: {result.inserted_count} documents')

# Verifikasi data yang diinsert
print("\n=== Data Courses ===")
for course in courses_collection.find():
    print(f"Course: {course['course']}, Enrollments: {course['enrollments']}, Department: {course['department']}")

Courses inserted successfully. Total: 4 documents

=== Data Courses ===
Course: Math 101, Enrollments: 30, Department: Mathematics
Course: CS 102, Enrollments: 25, Department: Computer Science
Course: History 201, Enrollments: 20, Department: History
Course: Physics 202, Enrollments: 15, Department: Physics


In [2]:
# Query for courses with enrollments greater than 20
print("=== Courses dengan Enrollments > 20 ===")
cursor = courses_collection.find({'enrollments': {'$gt': 20}})
count = 0
for course in cursor:
    print(f"Course: {course['course']}, Enrollments: {course['enrollments']}")
    count += 1
print(f"Total courses found: {count}")

=== Courses dengan Enrollments > 20 ===
Course: Math 101, Enrollments: 30
Course: CS 102, Enrollments: 25
Total courses found: 2


In [3]:
# Query courses in Computer Science or Mathematics departments
print("\n=== Courses dari Computer Science atau Mathematics ===")
cursor = courses_collection.find({
    'department': {'$in': ['Computer Science', 'Mathematics']}
})
count = 0
for course in cursor:
    print(f"Course: {course['course']}, Department: {course['department']}")
    count += 1
print(f"Total courses found: {count}")


=== Courses dari Computer Science atau Mathematics ===
Course: Math 101, Department: Mathematics
Course: CS 102, Department: Computer Science
Total courses found: 2


In [4]:
# Average enrollment per department using aggregation
print("\n=== Average Enrollment per Department ===")
pipeline = [
    {
        '$group': {
            '_id': '$department', 
            'average_enrollment': {'$avg': '$enrollments'}
        }
    },
    {
        '$sort': {'average_enrollment': -1}  # Sort descending
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Average Enrollment: {result['average_enrollment']:.2f}")


=== Average Enrollment per Department ===
Department: Mathematics, Average Enrollment: 30.00
Department: Computer Science, Average Enrollment: 25.00
Department: History, Average Enrollment: 20.00
Department: Physics, Average Enrollment: 15.00


In [5]:
# Maximum enrollment per department
print("\n=== Maximum Enrollment per Department ===")
pipeline = [
    {
        '$group': {
            '_id': '$department', 
            'max_enrollment': {'$max': '$enrollments'}
        }
    },
    {
        '$sort': {'max_enrollment': -1}
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Max Enrollment: {result['max_enrollment']}")


=== Maximum Enrollment per Department ===
Department: Mathematics, Max Enrollment: 30
Department: Computer Science, Max Enrollment: 25
Department: History, Max Enrollment: 20
Department: Physics, Max Enrollment: 15


In [6]:
# Projection to rename fields
print("\n=== Renamed Fields using $project ===")
pipeline = [
    {
        '$project': {
            '_id': 0,  # Exclude _id field
            'course_name': '$course', 
            'department_name': '$department', 
            'enrollments': 1  # Include as is
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Course: {result['course_name']}, Department: {result['department_name']}, Enrollments: {result['enrollments']}")


=== Renamed Fields using $project ===
Course: Math 101, Department: Mathematics, Enrollments: 30
Course: CS 102, Department: Computer Science, Enrollments: 25
Course: History 201, Department: History, Enrollments: 20
Course: Physics 202, Department: Physics, Enrollments: 15


In [7]:
# Adding enrollment category field based on enrollments
print("\n=== Courses dengan Enrollment Category ===")
pipeline = [
    {
        '$addFields': {
            'enrollment_category': {
                '$cond': {
                    'if': {'$gt': ['$enrollments', 20]}, 
                    'then': 'high', 
                    'else': 'low'
                }
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'course': 1,
            'enrollments': 1,
            'enrollment_category': 1
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Course: {result['course']}, Enrollments: {result['enrollments']}, Category: {result['enrollment_category']}")


=== Courses dengan Enrollment Category ===
Course: Math 101, Enrollments: 30, Category: high
Course: CS 102, Enrollments: 25, Category: high
Course: History 201, Enrollments: 20, Category: low
Course: Physics 202, Enrollments: 15, Category: low


In [8]:
print("\n=== HOMEWORK 1: Count Courses per Department ===")
pipeline = [
    {
        '$group': {
            '_id': '$department',
            'course_count': {'$sum': 1}  # Count documents
        }
    },
    {
        '$sort': {'course_count': -1}
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Total Courses: {result['course_count']}")


=== HOMEWORK 1: Count Courses per Department ===
Department: History, Total Courses: 1
Department: Physics, Total Courses: 1
Department: Mathematics, Total Courses: 1
Department: Computer Science, Total Courses: 1


In [11]:
print("\n=== HOMEWORK 2: CS Courses with Enrollments > 25 ===")

# First, let's add more CS courses for better demonstration
additional_courses = [
    InsertOne({
        'course': 'CS 201', 
        'enrollments': 28, 
        'department': 'Computer Science'
    }),
    InsertOne({
        'course': 'CS 301', 
        'enrollments': 22, 
        'department': 'Computer Science'
    })
]
courses_collection.bulk_write(additional_courses)

# Now perform the aggregation
pipeline = [
    {
        '$match': {
            'department': 'Computer Science',
            'enrollments': {'$gt': 25}
        }
    },
    {
        '$group': {
            '_id': '$department',
            'total_courses': {'$sum': 1},
            'total_enrollments': {'$sum': '$enrollments'},
            'avg_enrollments': {'$avg': '$enrollments'}
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(f"Department: {result['_id']}")
    print(f"  Total Courses: {result['total_courses']}")
    print(f"  Total Enrollments: {result['total_enrollments']}")
    print(f"  Average Enrollments: {result['avg_enrollments']:.2f}")

# Show individual courses
print("\nDetailed Courses:")
for course in courses_collection.find({
    'department': 'Computer Science',
    'enrollments': {'$gt': 25}
}):
    print(f"  - {course['course']}: {course['enrollments']} students")


=== HOMEWORK 2: CS Courses with Enrollments > 25 ===
Department: Computer Science
  Total Courses: 3
  Total Enrollments: 84
  Average Enrollments: 28.00

Detailed Courses:
  - CS 201: 28 students
  - CS 201: 28 students
  - CS 201: 28 students


In [12]:
print("\n=== HOMEWORK 3: Join Courses with Students using $lookup ===")

# Create students collection with enrollments
students_collection = db['students']
students_collection.delete_many({})  # Clear existing

students_data = [
    {
        'student_id': 'S001',
        'name': 'Alice Johnson',
        'enrolled_course': 'Math 101',
        'grade': 'A'
    },
    {
        'student_id': 'S002',
        'name': 'Bob Smith',
        'enrolled_course': 'Math 101',
        'grade': 'B+'
    },
    {
        'student_id': 'S003',
        'name': 'Charlie Brown',
        'enrolled_course': 'CS 102',
        'grade': 'A-'
    },
    {
        'student_id': 'S004',
        'name': 'Diana Prince',
        'enrolled_course': 'CS 102',
        'grade': 'A'
    },
    {
        'student_id': 'S005',
        'name': 'Eve Adams',
        'enrolled_course': 'History 201',
        'grade': 'B'
    }
]

students_collection.insert_many(students_data)
print(f"Inserted {len(students_data)} students")

# Perform $lookup to join collections
pipeline = [
    {
        '$lookup': {
            'from': 'students',  # Collection to join
            'localField': 'course',  # Field from courses
            'foreignField': 'enrolled_course',  # Field from students
            'as': 'enrolled_students'  # Output array field
        }
    },
    {
        '$addFields': {
            'student_count': {'$size': '$enrolled_students'}
        }
    },
    {
        '$project': {
            '_id': 0,
            'course': 1,
            'department': 1,
            'enrollments': 1,
            'student_count': 1,
            'enrolled_students': {
                'name': 1,
                'grade': 1
            }
        }
    }
]

print("\n=== Courses with Enrolled Students ===")
for result in courses_collection.aggregate(pipeline):
    print(f"\nCourse: {result['course']}")
    print(f"Department: {result['department']}")
    print(f"Capacity: {result['enrollments']} students")
    print(f"Actually Enrolled: {result['student_count']} students")
    if result['enrolled_students']:
        print("Students:")
        for student in result['enrolled_students']:
            print(f"  - {student['name']} (Grade: {student['grade']})")
    else:
        print("  No students enrolled yet")


=== HOMEWORK 3: Join Courses with Students using $lookup ===
Inserted 5 students

=== Courses with Enrolled Students ===

Course: Math 101
Department: Mathematics
Capacity: 30 students
Actually Enrolled: 2 students
Students:
  - Alice Johnson (Grade: A)
  - Bob Smith (Grade: B+)

Course: CS 102
Department: Computer Science
Capacity: 25 students
Actually Enrolled: 2 students
Students:
  - Charlie Brown (Grade: A-)
  - Diana Prince (Grade: A)

Course: History 201
Department: History
Capacity: 20 students
Actually Enrolled: 1 students
Students:
  - Eve Adams (Grade: B)

Course: Physics 202
Department: Physics
Capacity: 15 students
Actually Enrolled: 0 students
  No students enrolled yet

Course: CS 201
Department: Computer Science
Capacity: 28 students
Actually Enrolled: 0 students
  No students enrolled yet

Course: CS 301
Department: Computer Science
Capacity: 22 students
Actually Enrolled: 0 students
  No students enrolled yet

Course: CS 201
Department: Computer Science
Capacity: 28 